# SQL-практикум: аналитика продаж

In [1]:
!pip install -q duckdb
!pip install openpyxl

import pandas as pd
import duckdb

## Тестовое задание №1

In [2]:
task_1 = pd.read_excel(r'C:\Users\*\Downloads\*.xlsx', engine='openpyxl')

In [3]:
task_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_id    51 non-null     int64         
 1   order_date  51 non-null     datetime64[ns]
 2   channel     51 non-null     object        
 3   product     51 non-null     object        
 4   amount      51 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 2.1+ KB


### 1. Общая выручка за июль 2025

In [4]:
# создаем подключение
con = duckdb.connect(database=':memory:')

# регистрируем датафрейм в DuckDB под именем task_1
con.register('task_1', task_1)

# SQL-запрос для подсчёта суммарной выручки по всем заказам в выбранном месяце
query_1 = """
WITH
t_j AS (SELECT *
        FROM task_1
        WHERE DATE_TRUNC('month', order_date::date) = '2025-07-01')
SELECT SUM(amount) AS total_amount
FROM t_j
"""

# выполняем запрос и возвращаем результат в датафрейм
t1_q1 = con.execute(query_1).fetchdf()

# посмотрим на результат
t1_q1

,total_amount
0,350200.0


### 2. Выручка по каналам за июль 2025

In [5]:
# SQL-запрос с группировкой по каналам продаж и сортировкой по убыванию
query_2 = """
WITH
t_j AS (SELECT *
        FROM task_1
        WHERE DATE_TRUNC('month', order_date::date) = '2025-07-01')
SELECT SUM(amount)::int AS amount,
       channel
FROM t_j
GROUP BY channel
ORDER BY SUM(amount) DESC
"""

# выполняем запрос и возвращаем результат в датафрейм
t1_q2 = con.execute(query_2).fetchdf()

# посмотрим на результат
t1_q2

,amount,channel
0,210000,Реферал
1,71500,Телефон
2,40700,Email
3,28000,Сайт


### 3. Число уникальных заказов за июль 2025

In [6]:
# SQL-запрос для подсчёта уникальных `order_id` за выбранный месяц
query_3 = """
WITH
t_j AS (SELECT *
        FROM task_1
        WHERE DATE_TRUNC('month', order_date::date) = '2025-07-01')
SELECT count(amount) AS order_count, -- если бы поле не было ключом - тогда DISTINCT
FROM t_j
"""

# выполняем запрос и возвращаем результат в датафрейм
t1_q3 = con.execute(query_3).fetchdf()

# посмотрим на результат
t1_q3

,order_count
0,15


## Тестовое задание №2 

In [7]:
# создадим переменные для каждой таблицы
task_2_orders = pd.read_excel(r'C:\Users\*\Downloads\*.xlsx', engine='openpyxl', sheet_name='orders')
task_2_order_items = pd.read_excel(r'C:\Users\*\Downloads\*.xlsx', engine='openpyxl', sheet_name='order_items')
task_2_manager_targets = pd.read_excel(r'C:\Users\*\Downloads\*.xlsx', engine='openpyxl', sheet_name='manager_targets')

In [8]:
# регистрируем датафреймы в DuckDB 
con.register('orders', task_2_orders)
con.register('order_items', task_2_order_items)
con.register('manager_targets', task_2_manager_targets);

### 1. Выручка по месяцам и по каналам

In [9]:
# SQL-запрос для подсчёта выручки по месяцам
query_1_1 = """
WITH
order_product_amount AS (SELECT *,
                                qty * price AS amount
                         FROM order_items),
                         
order_amount AS (SELECT order_id,
                        SUM(amount) AS amount
                 FROM order_product_amount
                 GROUP BY order_id)
                 
SELECT strftime(DATE_TRUNC('month', (o.order_date)::date), '%Y-%m')  AS month,
       SUM(amount)::int AS amount
FROM orders AS o LEFT JOIN order_amount AS o_a ON o.order_id=o_a.order_id
GROUP BY DATE_TRUNC('month', o.order_date::date)
ORDER BY month
"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q1_1 = con.execute(query_1_1).fetchdf()

# посмотрим на результат
t2_q1_1

,month,amount
0,2025-07,373700
1,2025-08,234400
2,2025-09,184600


In [10]:
# SQL-запрос для подсчёта выручки по каналам
query_1_2 = """
WITH
order_product_amount AS (SELECT *,
                                qty * price AS amount
                         FROM order_items),
                         
order_amount AS (SELECT order_id,
                        SUM(amount) AS amount
                 FROM order_product_amount
                 GROUP BY order_id)
                 
SELECT strftime(DATE_TRUNC('month', (o.order_date)::date), '%Y-%m') AS month,
       o.channel,
       SUM(amount)::int AS amount
       
FROM orders AS o LEFT JOIN order_amount AS o_a ON o.order_id=o_a.order_id
GROUP BY strftime(DATE_TRUNC('month', o.order_date::date), '%Y-%m'), o.channel
ORDER BY month
"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q1_2 = con.execute(query_1_2).fetchdf()

# посмотрим на результат
t2_q1_2

,month,channel,amount
0,2025-07,Email,37200
1,2025-07,Реферал,210000
2,2025-07,Сайт,55000
3,2025-07,Телефон,71500
4,2025-08,Реферал,108000
5,2025-08,Email,38900
6,2025-08,Телефон,41500
7,2025-08,Сайт,46000
8,2025-09,Телефон,42000
9,2025-09,Сайт,13500


### 2. ТОП-3 продукта по выручке в каждом месяце

In [11]:
# SQL-запрос с ранжированием продуктов по выручке и выбором первых трёх
query_2_1 = """
WITH
o_i AS (SELECT *,
               qty * price AS amount
        FROM order_items),
        
m_p_a AS (SELECT DATE_TRUNC('month', (o.order_date)::date) AS month,
                 o_i.product,
                 SUM(amount) AS amount
          FROM orders AS o RIGHT JOIN o_i ON o.order_id=o_i.order_id
          GROUP BY DATE_TRUNC('month', (o.order_date)::date), o_i.product),
          
top AS (SELECT *,
               ROW_NUMBER() OVER(PARTITION BY month ORDER BY amount DESC) AS top_amount 
        FROM m_p_a
        ORDER BY month, top_amount)
        
SELECT strftime(month, '%Y-%m') AS month,
       product,
       amount
FROM top 
WHERE top_amount < 4
ORDER BY month, top_amount
"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q2_1 = con.execute(query_2_1).fetchdf()

# посмотрим на результат
t2_q2_1

,month,product,amount
0,2025-07,Интеграция с 1С,210000.0
1,2025-07,Внедрение Битрикс24,60500.0
2,2025-07,CRM Лицензия,45000.0
3,2025-08,Интеграция с 1С,108000.0
4,2025-08,CRM Лицензия,36000.0
5,2025-08,Техподдержка (мес.),31000.0
6,2025-09,Интеграция с 1С,110000.0
7,2025-09,Внедрение Битрикс24,31000.0
8,2025-09,Техподдержка (мес.),21000.0


### 3. Выручка по менеджерам и выполнение планов

In [12]:
# SQL-запрос с расчётом выручки, целевых показателей и процента выполнения
query_3_1 = """
WITH
a AS (SELECT *,
             qty * price AS amount
      FROM order_items),

o_i_a AS (SELECT order_id,
                 SUM(amount) AS amount
          FROM a
          GROUP BY order_id),

o_a_m AS (SELECT strftime(order_date::date, '%Y-%m') AS year_month,
                 manager,
                 SUM(amount) AS amount,
          FROM orders AS o LEFT JOIN o_i_a ON o_i_a.order_id=o.order_id
          GROUP BY manager, strftime(order_date::date, '%Y-%m')
          ORDER BY strftime(order_date::date, '%Y-%m'), manager),

m_r AS (SELECT *, ROW_NUMBER() OVER(ORDER BY year_month, manager) AS rank_m
            FROM o_a_m),

m_t AS (SELECT *, ROW_NUMBER() OVER(ORDER BY month, manager) AS rank_m
        FROM manager_targets),
        
m_p AS (SELECT m_t.month,
               m_r.manager,
               m_r.amount::int AS amount,
               m_t.target_revenue,
               ROUND(m_r.amount / m_t.target_revenue * 100, 1)::varchar || '%' AS real_prc
        FROM m_r LEFT JOIN m_t ON m_r.rank_m=m_t.rank_m)

SELECT month,
       LTRIM(manager || ': ' || amount || '; план ' || target_revenue || '; выполнение ' || real_prc) AS plan
FROM m_p
ORDER BY month

"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q3_1 = con.execute(query_3_1).fetchdf()

# посмотрим на результат
t2_q3_1

,month,plan
0,2025-07,Анна: 265000; план 120000; выполнение 220.8%
1,2025-07,Ирина: 71500; план 30000; выполнение 238.3%
2,2025-07,Сергей: 37200; план 30000; выполнение 124.0%
3,2025-08,Анна: 154000; план 140000; выполнение 110.0%
4,2025-08,Ирина: 41500; план 45000; выполнение 92.2%
5,2025-08,Сергей: 38900; план 40000; выполнение 97.3%
6,2025-09,Анна: 123500; план 130000; выполнение 95.0%
7,2025-09,Ирина: 42000; план 42000; выполнение 100.0%
8,2025-09,Сергей: 19100; план 30000; выполнение 63.7%


In [14]:
# SQL-запрос с расчётом доли менеджера в общей месячной выручке
query_3_2 = """
WITH
a AS (SELECT *,
             qty * price AS amount
      FROM order_items),

o_i_a AS (SELECT order_id,
                 SUM(amount) AS amount
          FROM a
          GROUP BY order_id),

o_a_m AS (SELECT strftime(order_date::date, '%Y-%m') AS year_month,
                 manager,
                 SUM(amount) AS amount,
          FROM orders AS o LEFT JOIN o_i_a ON o_i_a.order_id=o.order_id
          GROUP BY manager, strftime(order_date::date, '%Y-%m')
          ORDER BY strftime(order_date::date, '%Y-%m'), manager),

m_s AS (SELECT *,
               SUM(amount) OVER(PARTITION BY year_month) AS month_sum
        FROM o_a_m)

SELECT year_month,
       manager,
       'доля менеджера в месячной выручке: ' || ROUND(amount / month_sum * 100, 1)::varchar || '%' AS prc_of_month
FROM m_s

"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q3_2 = con.execute(query_3_2).fetchdf()

# посмотрим на результат
t2_q3_2

,year_month,manager,prc_of_month
0,2025-09,Сергей,доля менеджера в месячной выручке: 10.3%
1,2025-09,Анна,доля менеджера в месячной выручке: 66.9%
2,2025-09,Ирина,доля менеджера в месячной выручке: 22.8%
3,2025-07,Анна,доля менеджера в месячной выручке: 70.9%
4,2025-07,Ирина,доля менеджера в месячной выручке: 19.1%
5,2025-07,Сергей,доля менеджера в месячной выручке: 10.0%
6,2025-08,Анна,доля менеджера в месячной выручке: 65.7%
7,2025-08,Ирина,доля менеджера в месячной выручке: 17.7%
8,2025-08,Сергей,доля менеджера в месячной выручке: 16.6%


### 4. Средний чек по каналам в разрезе месяцев

In [15]:
# SQL-запрос для подсчёта среднего чека по каждому каналу и месяцу
query_4 = """
WITH
o_i AS (SELECT order_id,
               SUM(qty * price) AS amount
        FROM order_items
        GROUP BY order_id)
SELECT strftime(DATE_TRUNC('month', order_date::date), '%Y-%m') AS month,
       AVG(amount)::int AS amount
FROM orders AS o LEFT JOIN o_i ON o_i.order_id=o.order_id
GROUP BY DATE_TRUNC('month', order_date::date), channel
ORDER BY month ASC, channel ASC
"""

# выполняем запрос и возвращаем результат в датафрейм
t2_q4 = con.execute(query_4).fetchdf()

# посмотрим на результат
t2_q4

,month,amount
0,2025-07,9300
1,2025-07,52500
2,2025-07,13750
3,2025-07,23833
4,2025-08,12967
5,2025-08,54000
6,2025-08,15333
7,2025-08,20750
8,2025-09,9550
9,2025-09,55000
